<a href="https://colab.research.google.com/github/jbcgames/Lab_1_Controls/blob/main/ResumenMetodo_Ziegler_Nichols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identificación de Modelo FOPDT mediante Ziegler-Nichols

# Introducción y Objetivos

En este trabajo realizamos la identificación de parámetros para un sistema térmico utilizando el método de Ziegler-Nichols. El sistema consiste en un calentador controlado por PWM y sensores de temperatura LM35. Nuestro objetivo fue obtener un modelo FOPDT (First Order Plus Dead Time) que describa adecuadamente la dinámica del sistema para futuros diseños de controladores.

##. Metodología y Explicación Detallada del Código

   Configuración Inicial y Lectura de Datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/jbcgames/Lab_1_Controls/main/Datos.txt')
tiempo = df['Tiempo (s)'].values
T1 = df[' T1 (C)'].values
pwm = df[' PWM_Heater1'].values

Explicación:

Leemos los datos directamente desde un repositorio GitHub donde están almacenados los resultados experimentales.

Extraemos tres vectores importantes:

-tiempo: Contiene los instantes de tiempo de las mediciones.

-T1: Valores de temperatura medidos por el sensor LM35.

-pwm: Señal de control aplicada al calentador (ciclo de trabajo PWM).

##Detección del Escalón de Entrada

In [ ]:
indices_escalon = np.where((pwm[:-1] == 0) & (pwm[1:] > 0))[0]
if len(indices_escalon) == 0:
    print("No se detectó un escalón en la señal PWM.")
    exit()
idx_escalon = indices_escalon[0]
t_escalon = tiempo[idx_escalon + 1]
print(f"Escalón detectado en t = {t_escalon:.2f} s")

Explicación:

-Buscamos el punto donde la señal PWM cambia de 0 a un valor positivo (aplicación del escalón).

-np.where identifica los índices donde ocurre esta transición.

-Verificamos que efectivamente exista un escalón en los datos.

-Calculamos el tiempo exacto donde se aplica el escalón, fundamental para el análisis posterior.

##Preprocesamiento de Datos

In [ ]:
mask = (tiempo <= 450)
tiempo = tiempo[mask]
T1 = T1[mask]
pwm = pwm[mask]

Explicación:

-Filtramos los datos para considerar solo los primeros 450 segundos, donde se observa la respuesta completa al escalón.

-Esto elimina datos innecesarios que podrían afectar el análisis.

##Suavizado de la Señal de Temperatura

In [ ]:
window_length = 51
polyorder = 3
T1_smooth = savgol_filter(T1, window_length, polyorder)

Explicación:

-Aplicamos un filtro Savitzky-Golay para reducir el ruido en la señal de temperatura.

-window_length=51: El tamaño de la ventana de suavizado (debe ser impar).

-polyorder=3: El orden del polinomio usado para el ajuste local.

-Este paso es crucial para obtener una derivada más limpia y detectar correctamente el punto de inflexión.

##Cálculo de Parámetros Iniciales

In [ ]:
y0 = T1[0]
yf = T1[-1]
delta_y = yf - y0
pwm_step = pwm[0] if pwm[0] > 0 else pwm[np.where(pwm > 0)[0][0]]
Kp = delta_y / pwm_step

Explicación:

-y0: Temperatura inicial antes del escalón.

-yf: Temperatura en estado estable después del escalón.

-delta_y: Cambio total en la temperatura.

-Kp: Ganancia del proceso, calculada como el cambio en la salida dividido por el cambio en la entrada (PWM).

##Determinación del Punto de Inflexión

In [ ]:
dT1_dt_smooth = np.gradient(T1_smooth, tiempo)
idx_max_slope = np.argmax(dT1_dt_smooth)
t_inflex = tiempo[idx_max_slope]
y_inflex = T1_smooth[idx_max_slope]
max_slope = dT1_dt_smooth[idx_max_slope]

Explicación:

-Calculamos la derivada de la señal suavizada para encontrar la pendiente máxima.

-np.gradient aproxima numéricamente la derivada.

-np.argmax encuentra el índice de la pendiente máxima (punto de inflexión).

-Obtenemos el tiempo, valor de temperatura y pendiente en este punto crítico.

##Cálculo de Tiempo Muerto y Constante de Tiempo

In [ ]:
t_dead = t_inflex + (y0 - y_inflex)/max_slope
t_end = t_inflex + (yf - y_inflex)/max_slope
tm = t_dead - t_escalon
tau = t_end - t_dead

Explicación:

-t_dead: Intersección de la tangente en el punto de inflexión con la línea de temperatura inicial.

-t_end: Intersección de la misma tangente con la temperatura final.

-tm: Tiempo muerto (diferencia entre aplicación del escalón y t_dead).

-tau: Constante de tiempo del sistema (diferencia entre t_end y t_dead).

##Simulación del Modelo FOPDT

In [ ]:
t_modelo = tiempo.copy()
respuesta_modelo = np.zeros_like(t_modelo)
for i, t in enumerate(t_modelo):
    if t < (t_escalon + tm):
        respuesta_modelo[i] = y0
    else:
        respuesta_modelo[i] = y0 + Kp * pwm_step * (1 - np.exp(-(t - (t_escalon + tm)) / tau))

Explicación detallada:

-Generamos la respuesta teórica del modelo FOPDT obtenido.

-Para tiempos menores que t_escalon + tm, la salida permanece en y0.

-Luego, sigue una exponencial creciente característica de sistemas de primer orden.

##Visualización de Resultados

In [ ]:
plt.figure()
plt.plot(tiempo, T1, label='Datos originales')
plt.plot(tiempo, T1_smooth, label='Datos suavizados (Savitzky-Golay)')
plt.plot(t_modelo, respuesta_modelo, '--', label='Modelo Z-N FOPDT (filtrado)')
plt.axvline(t_escalon, color='orange', linestyle='--', label='Inicio escalón')
plt.axvline(t_escalon + tm, color='gray', linestyle='--', label='Tiempo muerto')
plt.xlabel('Tiempo [s]')
plt.ylabel('Temperatura [°C]')
plt.title('Método Tangente Ziegler-Nichols con Datos Suavizados')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

Explicación detallada:

Graficamos tres curvas principales:

-Datos originales de temperatura.

-Datos suavizados.

Respuesta del modelo FOPDT obtenido:

-Líneas verticales marcan el inicio del escalón y el tiempo muerto.

-La gráfica permite comparar visualmente el ajuste del modelo a los datos reales.

##Resultados Obtenidos

La gráfica muestra que el modelo FOPDT se ajusta razonablemente bien a los datos experimentales, particularmente en la parte inicial de la respuesta y en el estado estable. Se observa cierta discrepancia en la región intermedia, lo que podría deberse a:

No linealidades no consideradas en el modelo FOPDT

Efectos de acoplamiento térmico con otros elementos del sistema

Limitaciones del método de Ziegler-Nichols para sistemas con dinámicas más complejas

##Conclusiones

El método de Ziegler-Nichols aplicado a la respuesta al escalón permitió obtener un modelo FOPDT con parámetros físicamente razonables para el sistema térmico, donde el tiempo muerto identificado (10.33 s) resulta coherente con la inercia térmica esperada en este tipo de sistemas, evidenciando el retardo necesario para que el calor se propague desde el calentador al sensor, mientras que la constante de tiempo relativamente grande (171.74 s) confirma la naturaleza lenta del proceso, característica típica de sistemas térmicos con alta capacidad calorífica, destacando además que el suavizado de datos fue esencial para una identificación precisa del punto de inflexión y el cálculo correcto de la pendiente máxima.